In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
checker = data["Exited"].value_counts()
checker

0    7963
1    2037
Name: Exited, dtype: int64

In [5]:
x = data.drop("Exited", axis = 1)
y = data["Exited"]

In [6]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler , OneHotEncoder ,LabelEncoder ,MinMaxScaler

In [7]:
sampling = RandomOverSampler(sampling_strategy="auto", random_state=0)
x_sampler, y_sampler = sampling.fit_resample(x, y)

In [8]:
scaler = StandardScaler()
encoder = OneHotEncoder()

In [9]:
x.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [10]:
from sklearn.compose import ColumnTransformer
category =["Surname","Geography","Gender","NumOfProducts","HasCrCard","IsActiveMember"]
numerical = ["CreditScore","RowNumber", "CustomerId", "Age","Tenure","Balance","EstimatedSalary"]


transform = ColumnTransformer([("numerical",scaler,numerical),
                               ("category",encoder,category)],remainder="passthrough")

trans_x =  transform.fit_transform(x_sampler)
trans_x

<15926x2952 sparse matrix of type '<class 'numpy.float64'>'
	with 207038 stored elements in Compressed Sparse Row format>

In [11]:
trans_x.shape

(15926, 2952)

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(42)
x_train ,x_test ,y_train ,y_test = train_test_split(trans_x , y_sampler,test_size =0.2)

In [13]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [14]:
model = Sequential()

model.add(Dense(units = 200 , activation = "relu" ,input_shape=(2952,)))

model.add(Dense(units = 68 ,activation = "relu"))

model.add(Dense(2 ,activation = "softmax"))

model.compile(optimizer='adam', loss="binary_crossentropy",metrics = ["accuracy"])


In [18]:
from keras.utils import to_categorical

y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test) 

model.fit(x_train ,y_train_encoded ,batch_size = 32 ,epochs = 3)

Epoch 1/3
399/399 [==============================] - 5s 13ms/step - loss: 0.0827 - accuracy: 0.9721
Epoch 2/3
399/399 [==============================] - 6s 15ms/step - loss: 0.0407 - accuracy: 0.9875
Epoch 3/3
399/399 [==============================] - 5s 13ms/step - loss: 0.0210 - accuracy: 0.9947


In [19]:
y_predict_encoded = model.predict(x_test)
y_pred = np.argmax(y_predict_encoded,axis = 1)
y_pred

100/100 [==============================] - 0s 2ms/step


array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("\nAccuracy:", accuracy)

# Precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

# Recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

# F1-Score
f1 = f1_score(y_test, y_pred)
print("F1-Score:", f1)

# ROC-AUC Score (only applicable for binary classification tasks)
roc_auc = roc_auc_score(y_test, y_pred)
print("ROC-AUC Score:", roc_auc)

Confusion Matrix:
 [[1498  135]
 [  56 1497]]

Accuracy: 0.9400502197112367
Precision: 0.9172794117647058
Recall: 0.9639407598197038
F1-Score: 0.9400313971742543
ROC-AUC Score: 0.9406354135901949
